# Explore the two datasources - Traffic violations

For the same topic, two different versions of the same dataset were identified, which were both published on openML. 

The first is unprocessed and can be found [here](https://api.openml.org/d/42132). The second one is a preprocessed and subsampled version that can be downloaded [here](https://www.openml.org/search?type=data&status=active&sort=runs&order=desc&id=42345). 

In [ ]:
import datetime
import matplotlib.pyplot as plt
import nltk
import pandas as pd
import seaborn as sns
import spacy
import string

from nltk.corpus import stopwords
from pathlib import Path
from typing import List, Tuple
from scipy.io.arff import loadarff
from scipy.io.arff._arffread import MetaData
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
FULL_DATA_PATH = Path("../data/dataset.csv")
DATA_PATH = Path("../data/file65ef3a759daf.arff")

## Preprocessed dataset

In [ ]:
data, meta = loadarff(DATA_PATH)
data = pd.DataFrame(data)

In [ ]:
print("Dimensions of the dataset:", data.shape)

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
# remove b string from data
str_df = data.select_dtypes([object])
str_df = str_df.stack().str.decode("utf-8").unstack()
data = pd.concat([str_df, data.select_dtypes(exclude=[object])], axis=1)

In [ ]:
def load_data(path_: Path) -> Tuple[pd.DataFrame, MetaData]:
    """Loads the .arff file (incl. metadata) and converts to utf-8.

    Parameters
    -------
    path_ : Path
            Path of the data.

    Returns
    -------
    data : pd.DataFrame
            Data as a dataframe.
    meta : scipy.io.arff._arffread.Metadata
            Metadata of the dataset.
    """
    # load df and metadata from .arff
    data, meta = loadarff(path_)
    data = pd.DataFrame(data)

    # remove b string from data
    str_df = data.select_dtypes([object])
    str_df = str_df.reset_index().melt(id_vars="index").set_index("index")
    str_df["value"] = str_df["value"].str.decode("utf-8")

    # rename the 'value' column to avoid conflicts and perform pivot
    str_df = str_df.rename(columns={"value": "decoded_value"})
    str_df = pd.pivot_table(
        str_df, columns="variable", values="decoded_value", index="index", aggfunc=lambda x: x
    )

    # reset both the column and index names to None
    str_df = str_df.rename_axis(index=None, columns=None)

    # merge str and non-str columns
    data = pd.concat([str_df, data.select_dtypes(exclude=[object])], axis=1)

    return data, meta

In [ ]:
def change_to_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """Change yes/no values in columns to 0/1.

    Parameters
    -------
    df : pd.DataFrame
            Data to transform.

    Returns
    -------
    df : pd.DataFrame
            Transformed data.
    """
    for col in df.columns:
        # only change columns that have no missing values to 0 / 1
        if set(df[col].unique().tolist()) - set(["No", "Yes"]) == set():
            df[col] = df[col].map(dict(Yes=1, No=0))

    return df

In [ ]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    """Creates new features based and drops the old ones.

        - If the Vehicle's State is Maryland (MD)
        - If the Driver's State is Maryland (MD)
        - If the Driver License's State is Maryland (MD)

    Parameters
    -------
    df : pd.DataFrame
            Data to transform.

    Returns
    -------
    df : pd.DataFrame
            Transformed data.
    """
    # Vehicle's State
    df["State_MD"] = (df["State"] == "MD").astype(int)
    df.loc[df["State"] == "?", "State_MD"] = -1

    # Driver's State
    df["Driver_State_MD"] = (df["Driver.State"] == "MD").astype(int)
    df.loc[df["Driver.State"] == "?", "Driver_State_MD"] = -1

    # Driver License's State
    df["DL_State_MD"] = (df["DL.State"] == "MD").astype(int)
    df.loc[df["DL.State"] == "?", "DL_State_MD"] = -1

    df = df.drop(columns=["State", "Driver.State", "DL.State"])
    return df

In [ ]:
def transform_label(df: pd.DataFrame) -> pd.DataFrame:
    """Drops rows that are equal to SERO and changes label to "Citation" with 0/1 - values.

    Parameters
    -------
    df : pd.DataFrame
            Data to transform.

    Returns
    -------
    df : pd.DataFrame
            Transformed data.
    """
    df = df[df["Violation.Type"] != "SERO"].copy()
    df["Citation"] = df.loc[:, "Violation.Type"].apply(lambda x: 1 if x == "Citation" else 0)
    df = df.drop(columns=["Violation.Type"])
    return df

In [ ]:
def convert_float_to_int(df: pd.DataFrame, column_name: str = "Year") -> pd.DataFrame:
    """If possible to convert float to int converts to int.

    Parameters
    -------
    df : pd.DataFrame
            Data to transform.
    column_name : str
            Column that should be converted to int.

    Returns
    -------
    df : pd.DataFrame
            Transformed data.
    """
    assert (
        df[column_name].dropna().apply(lambda x: x.is_integer()).all()
    ), "Can't be converted to int"
    df[column_name] = df[column_name].fillna(-1).astype(int)

    # get today's year and filter by -1 (na), above 1990 or below/equal today's year
    year = int(datetime.date.today().strftime("%Y"))
    df = df[(df[column_name] == -1) | ((df[column_name] > 1990) & (df[column_name] <= year))]

    return df

In [ ]:
def drop_cols(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """Drop columns from dataframe.

    Parameters
    -------
    df : pd.DataFrame
            Data to transform.
    cols : List
            List of columns to drop.

    Returns
    -------
    df : pd.DataFrame
            Transformed data.
    """
    df = df.drop(columns=cols)

    return df

In [ ]:
def filter_na(df: pd.DataFrame) -> pd.DataFrame:
    """Filter na-values in full df.

    Since analysis showed that values are missing at random
    across groups, make less than 1% of instances and instance normally has
    multiple missing feature values, this operation is valid.

    Parameters
    -------
    df : pd.DataFrame
            Data to transform.

    Returns
    -------
    df : pd.DataFrame
            Transformed data.
    """
    # filter out -1 in integer columns
    cols = df.select_dtypes([int]).columns
    for col in cols:
        df = df[df[col] != -1].copy()

    # filter out ? in string columns
    cols = df.select_dtypes([object]).columns
    for col in cols:
        df = df[~df[col].isin(["U", "?"])].copy()

    return df

In [ ]:
def preprocessor(data_path: Path, cols: List[str]) -> pd.DataFrame:
    """Load data and perform preprocessing steps.

    Parameters
    -------
    path_ : Path
            Path of the data.
    cols : List
            List of columns to drop.

    Returns
    -------
    data : pd.DataFrame
            Processed data.
    """
    # load the data
    data, _ = load_data(data_path)

    # convert yes/no to 0/1 and year to int
    data = change_to_numeric(data)
    data = convert_float_to_int(data)

    # perform feature engineering on state columns
    data = feature_engineering(data)

    # drop unwished cols
    data = drop_cols(data, cols)

    # transform label to 0/1 for citation
    data = transform_label(data)

    # filter na
    data = filter_na(data)

    return data

In [ ]:
data = preprocessor(
    DATA_PATH, ["Model", "Charge", "Driver.City", "Arrest.Type", "Commercial.Vehicle"]
)

In [ ]:
data.head()

In [ ]:
nltk.download("stopwords")
nlp = spacy.load("en_core_web_sm")

In [ ]:
def preprocess_text(df: pd.DataFrame, column_name: str = "Description") -> pd.DataFrame:
    """reformats text so that LDA can be applied in the next step

    Parameters
    -------
    df : pd.DataFrame
            Data to transform.
    column_name : (str, optional)
            Name of column to be transformed.
            Defaults to "Description".

    Returns
    -------
    df : pd.DataFrame
            Transformed data.
    """
    # making text lower case
    lowercase_text = df[column_name].apply(lambda x: x.lower())

    # tokenize text
    tokenized_text = lowercase_text.apply(lambda x: nltk.word_tokenize(x))

    # removing stopwords
    stop_words = set(stopwords.words("english"))
    clean_text = tokenized_text.apply(
        lambda tokens: [
            word for word in tokens if word not in stop_words and word not in string.punctuation
        ]
    )

    # lemmatize words
    lemmatized_text = clean_text.apply(
        lambda tokens: [token.lemma_ for token in nlp(" ".join(tokens))]
    )

    df["description_clean"] = lemmatized_text.apply(lambda lem_tokens: " ".join(lem_tokens))

    return df

In [ ]:
data = preprocess_text(data)

In [ ]:
data.head()

In [ ]:
# data['description_clean'] = data['Description'].apply(preprocess_text)

In [ ]:
# # Create a CountVectorizer
# vectorizer = CountVectorizer(max_features=1000, stop_words='english')
# X = vectorizer.fit_transform(data['description_clean'])

# # create an LDA model
# num_topics = 20
# lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)

# lda.fit(X)

# topic_distributions = lda.transform(X)

# for i in range(num_topics):
#     data[f"Topic_{i+1}"] = topic_distributions[:, i]

In [ ]:
def create_n_topics(
    df: pd.DataFrame,
    column_name: str = "description_clean",
    n_topics: int = 10,
    max_features: int = 1000,
):
    """
    Applies Latent Dirichlet Allocation (LDA) to a text column in a DataFrame and adds LDA topic distributions as new features.

    Parameters
    ----------
    df : pd.DataFrame
        Data to transform.

    column_name : str, optional
        Name of the column to be transformed. Defaults to "Description".

    num_topics : int, optional
        Number of topics for LDA. Defaults to 10.

    max_features : int, optional
        Maximum number of features for CountVectorizer. Defaults to 1000.

    Returns
    -------
    df : pd.DataFrame
        Transformed data with added LDA topic features.
    """
    # Create a CountVectorizer
    vectorizer = CountVectorizer(max_features=max_features, stop_words="english")
    X = vectorizer.fit_transform(data[column_name])

    # create an LDA model
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)

    lda.fit(X)

    topic_distributions = lda.transform(X)

    for i in range(n_topics):
        df[f"Topic_{i+1}"] = topic_distributions[:, i]

    return df

In [ ]:
data = create_n_topics(data)

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
df_numeric = data.select_dtypes(exclude=[object])
# shows correlation of numeric features
corr = df_numeric.corr()
corr

sns.heatmap(corr)
plt.title("Correlation plot of numerical features")

In [ ]:
def dummy_encoder(
    df: pd.DataFrame = None,
    cols: list = None,
) -> pd.DataFrame:
    """Encode the categorical variables within the dataframe.

    Parameters
    -------
    df : pd.DataFrame
            Dataframe to split.
    cols : list
            List with the categorical variables we wish to dummy encode.

    Returns
    -------
    df : pd.DataFrame
        Dataframe with categorical variables one-hot-encoded(original df if catboost=True).
    """
    df = pd.get_dummies(df, cols, dtype=int)
    return df